# 05 - Polars + Ray: En İyi İki Dünya

Bu notebook'ta **Polars** ve **Ray**'i birlikte kullanarak optimal bir pipeline oluşturacağız.

**Strateji:**
- **Polars**: Hızlı veri yükleme, temizleme ve feature engineering (5-10x Pandas)
- **Ray**: Paralel model training ve hyperparameter tuning

**Neden Bu Kombinasyon?**
- Polars: Single-node'da en hızlı DataFrame kütüphanesi
- Ray: Distributed computing ve ML paralellestirme
- Birlikte: End-to-end optimum performans

**Veri Seti:** NYC Yellow Taxi 2023 (12 ay, ~40M satır)

## 1. Kurulum

In [1]:
# Gerekli kütüphaneler
!pip install polars ray scikit-learn xgboost pyarrow -q

In [79]:
import polars as pl
import ray
import numpy as np
import time
import json
import os
import psutil
import gc
import urllib.request
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor

print(f"Polars version: {pl.__version__}")
print(f"Ray version: {ray.__version__}")
print(f"CPU count: {os.cpu_count()}")

Polars version: 1.31.0
Ray version: 2.52.1
CPU count: 12


In [80]:
# Benchmark fonksiyonları
results = {
    'framework': 'polars_ray_hybrid',
    'dataset': 'nyc_taxi_12_months',
    'operations': {}
}

def get_memory_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def benchmark(func, name):
    gc.collect()
    mem_before = get_memory_mb()
    start = time.time()
    result = func()
    end = time.time()
    mem_after = get_memory_mb()
    
    duration = end - start
    mem_used = mem_after - mem_before
    
    results['operations'][name] = {
        'duration_sec': round(duration, 3),
        'memory_mb': round(mem_used, 2)
    }
    
    print(f"\n{'='*50}")
    print(f"✓ {name}")
    print(f"  Süre: {duration:.3f} saniye")
    print(f"  Bellek: {mem_used:.2f} MB")
    print(f"{'='*50}")
    
    return result

In [81]:
# Veri indirme - 12 ay
DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)

MONTHS = ['2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12']
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{}.parquet"

taxi_files = []
total_size = 0

for month in MONTHS:
    filename = f"yellow_tripdata_{month}.parquet"
    filepath = os.path.join(DATA_DIR, filename)
    taxi_files.append(filepath)
    
    if not os.path.exists(filepath):
        url = BASE_URL.format(month)
        print(f"İndiriliyor: {filename}...")
        urllib.request.urlretrieve(url, filepath)
        print(f"İndirildi: {filename}")
    else:
        print(f"Mevcut: {filename}")
    
    total_size += os.path.getsize(filepath)

print(f"\nToplam dosya boyutu: {total_size / 1024**2:.1f} MB")
print(f"Dosya sayısı: {len(taxi_files)}")

Mevcut: yellow_tripdata_2023-01.parquet
Mevcut: yellow_tripdata_2023-02.parquet
Mevcut: yellow_tripdata_2023-03.parquet
Mevcut: yellow_tripdata_2023-04.parquet
Mevcut: yellow_tripdata_2023-05.parquet
Mevcut: yellow_tripdata_2023-06.parquet
Mevcut: yellow_tripdata_2023-07.parquet
Mevcut: yellow_tripdata_2023-08.parquet
Mevcut: yellow_tripdata_2023-09.parquet
Mevcut: yellow_tripdata_2023-10.parquet
Mevcut: yellow_tripdata_2023-11.parquet
Mevcut: yellow_tripdata_2023-12.parquet

Toplam dosya boyutu: 606.3 MB
Dosya sayısı: 12


## 2. Polars ile Hızlı Veri Yükleme (Lazy Mode)

In [82]:
# Polars Lazy: Sadece Ocak 2023 verisi (04 ile aynı veri)
def load_with_polars_lazy():
    # Sadece ilk dosyayı (Ocak 2023) yükle - 04 notebook ile aynı
    lf = pl.scan_parquet(taxi_files[0])  # Ocak 2023
    
    # Satır sayısını al
    row_count = lf.select(pl.len()).collect().item()

    return lf, row_count

lf, total_rows = benchmark(load_with_polars_lazy, 'polars_lazy_scan')
print(f"\nToplam satır: {total_rows:,}")
print(f"Schema: {lf.collect_schema()}")


✓ polars_lazy_scan
  Süre: 0.003 saniye
  Bellek: 0.00 MB

Toplam satır: 3,066,766
Schema: Schema({'VendorID': Int64, 'tpep_pickup_datetime': Datetime(time_unit='ns', time_zone=None), 'tpep_dropoff_datetime': Datetime(time_unit='ns', time_zone=None), 'passenger_count': Float64, 'trip_distance': Float64, 'RatecodeID': Float64, 'store_and_fwd_flag': String, 'PULocationID': Int64, 'DOLocationID': Int64, 'payment_type': Int64, 'fare_amount': Float64, 'extra': Float64, 'mta_tax': Float64, 'tip_amount': Float64, 'tolls_amount': Float64, 'improvement_surcharge': Float64, 'total_amount': Float64, 'congestion_surcharge': Float64, 'airport_fee': Float64})


In [83]:
# Polars ile hızlı istatistikler (lazy evaluation)
def polars_quick_stats():
    stats = lf.select([
        pl.col('fare_amount').mean().alias('avg_fare'),
        pl.col('trip_distance').mean().alias('avg_distance'),
        pl.col('tip_amount').mean().alias('avg_tip'),
        pl.col('total_amount').sum().alias('total_revenue'),
        pl.col('passenger_count').mean().alias('avg_passengers')
    ]).collect()
    return stats

stats = benchmark(polars_quick_stats, 'polars_aggregations')
print("\n12 Aylık Özet İstatistikler:")
print(stats)


✓ polars_aggregations
  Süre: 0.151 saniye
  Bellek: 0.05 MB

12 Aylık Özet İstatistikler:
shape: (1, 5)
┌───────────┬──────────────┬──────────┬───────────────┬────────────────┐
│ avg_fare  ┆ avg_distance ┆ avg_tip  ┆ total_revenue ┆ avg_passengers │
│ ---       ┆ ---          ┆ ---      ┆ ---           ┆ ---            │
│ f64       ┆ f64          ┆ f64      ┆ f64           ┆ f64            │
╞═══════════╪══════════════╪══════════╪═══════════════╪════════════════╡
│ 18.367069 ┆ 3.847342     ┆ 3.367941 ┆ 8.2865e7      ┆ 1.362532       │
└───────────┴──────────────┴──────────┴───────────────┴────────────────┘


## 3. Polars ile Feature Engineering

Polars'ın expression API'si ile hızlı feature engineering.

In [84]:
def polars_feature_engineering():
    """
    Polars ile feature engineering - 04 notebook ile AYNI feature'lar.
    """
    df = (
        lf
        # Sadece gerekli sütunları seç
        .select([
            'tpep_pickup_datetime',
            'trip_distance',
            'PULocationID',
            'DOLocationID', 
            'passenger_count',
            'fare_amount',
        ])
        # Filtreleme (04 ile aynı)
        .filter(
            (pl.col('fare_amount') > 0) &
            (pl.col('fare_amount') < 200) &
            (pl.col('trip_distance') > 0) &
            (pl.col('trip_distance') < 50) &
            (pl.col('passenger_count') > 0) &
            (pl.col('passenger_count').is_not_null())
        )
        # Feature extraction (04 ile AYNI)
        .with_columns([
            pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour'),
            pl.col('tpep_pickup_datetime').dt.weekday().alias('pickup_dayofweek'),
        ])
        # Derived features (04 ile AYNI)
        .with_columns([
            (pl.col('pickup_dayofweek') >= 5).cast(pl.Int8).alias('is_weekend'),
            (
                ((pl.col('pickup_hour') >= 7) & (pl.col('pickup_hour') <= 9)) |
                ((pl.col('pickup_hour') >= 17) & (pl.col('pickup_hour') <= 19))
            ).cast(pl.Int8).alias('is_rush_hour'),
        ])
        .collect()
    )
    return df

df_features = benchmark(polars_feature_engineering, 'polars_feature_engineering')
print(f"\nFeature DataFrame: {df_features.shape}")
print(f"Sütunlar: {df_features.columns}")


✓ polars_feature_engineering
  Süre: 0.150 saniye
  Bellek: 0.43 MB

Feature DataFrame: (2883426, 10)
Sütunlar: ['tpep_pickup_datetime', 'trip_distance', 'PULocationID', 'DOLocationID', 'passenger_count', 'fare_amount', 'pickup_hour', 'pickup_dayofweek', 'is_weekend', 'is_rush_hour']


In [85]:
# Veri önizleme
print("\nVeri Örneği:")
df_features.head(5)


Veri Örneği:


tpep_pickup_datetime,trip_distance,PULocationID,DOLocationID,passenger_count,fare_amount,pickup_hour,pickup_dayofweek,is_weekend,is_rush_hour
datetime[ns],f64,i64,i64,f64,f64,i8,i8,i8,i8
2023-01-01 00:32:10,0.97,161,141,1.0,9.3,0,7,1,0
2023-01-01 00:55:08,1.1,43,237,1.0,7.9,0,7,1,0
2023-01-01 00:25:04,2.51,48,238,1.0,14.9,0,7,1,0
2023-01-01 00:10:29,1.43,107,79,1.0,11.4,0,7,1,0
2023-01-01 00:50:34,1.84,161,137,1.0,12.8,0,7,1,0


In [86]:
# ML için sample al ve numpy'a çevir
# NOT: Pandas ile AYNI satırları seçmek için NumPy random kullanıyoruz
def prepare_ml_data():
    # 04 notebook ile AYNI yöntem: NumPy ile index seç
    np.random.seed(42)  # Pandas ile aynı seed
    sample_size = min(500_000, len(df_features))
    indices = np.random.choice(len(df_features), sample_size, replace=False)
    
    # Polars'ta iloc yerine gather kullan
    df_sample = df_features[indices]
    
    # 04 notebook ile AYNI feature'lar
    feature_cols = [
        'trip_distance', 'pickup_hour', 'pickup_dayofweek',
        'is_weekend', 'is_rush_hour', 
        'PULocationID', 'DOLocationID', 'passenger_count'
    ]
    
    X = df_sample.select(feature_cols).to_numpy()
    y = df_sample.select('fare_amount').to_numpy().ravel()
    
    return train_test_split(X, y, test_size=0.2, random_state=42), feature_cols

(X_train, X_test, y_train, y_test), feature_cols = benchmark(prepare_ml_data, 'prepare_ml_data')
print(f"\nTrain: {X_train.shape}")
print(f"Test: {X_test.shape}")
print(f"Features: {feature_cols}")


✓ prepare_ml_data
  Süre: 0.190 saniye
  Bellek: 45.76 MB

Train: (400000, 8)
Test: (100000, 8)
Features: ['trip_distance', 'pickup_hour', 'pickup_dayofweek', 'is_weekend', 'is_rush_hour', 'PULocationID', 'DOLocationID', 'passenger_count']


## 4. Ray Başlat

In [87]:
# Ray'i başlat
if ray.is_initialized():
    ray.shutdown()

ray.init(ignore_reinit_error=True)

print(f"\nRAY CLUSTER BİLGİSİ")
print("="*50)
print(f"Nodes: {len(ray.nodes())}")
print(f"CPUs: {ray.cluster_resources().get('CPU', 0)}")
print(f"Memory: {ray.cluster_resources().get('memory', 0) / 1024**3:.1f} GB")

2025-12-10 08:30:51,531	INFO worker.py:2023 -- Started a local Ray instance.



RAY CLUSTER BİLGİSİ
Nodes: 1
CPUs: 12.0
Memory: 58.1 GB


In [88]:
# Veriyi Ray object store'a yükle
X_train_ref = ray.put(X_train)
y_train_ref = ray.put(y_train)
X_test_ref = ray.put(X_test)
y_test_ref = ray.put(y_test)

print("Veri Ray object store'a yüklendi")

Veri Ray object store'a yüklendi


## 5. Ray ile Paralel Model Karşılaştırma

Farklı ML algoritmalarını paralel olarak eğitip karşılaştıracağız.

In [89]:
@ray.remote
def train_and_evaluate(model_name, model_params, X_train, y_train, X_test, y_test):
    """
    Bir modeli eğit ve değerlendir.
    Ray remote olarak paralel çalışır.
    """
    start = time.time()
    
    # Model seçimi
    if model_name == 'RandomForest':
        model = RandomForestRegressor(**model_params, random_state=42, n_jobs=1)
    elif model_name == 'GradientBoosting':
        model = GradientBoostingRegressor(**model_params, random_state=42)
    elif model_name == 'XGBoost':
        model = XGBRegressor(**model_params, random_state=42, n_jobs=1, verbosity=0)
    elif model_name == 'Ridge':
        model = Ridge(**model_params)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Eğitim
    model.fit(X_train, y_train)
    
    # Tahmin
    y_pred = model.predict(X_test)
    
    # Metrikler
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    duration = time.time() - start
    
    return {
        'model': model_name,
        'params': model_params,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'duration': duration
    }

In [90]:
# Model konfigürasyonları - 04 ile AYNI parametreler dahil
model_configs = [
    # Random Forest variants (04 ile aynı)
    ('RandomForest', {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 2}),
    ('RandomForest', {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2}),
    ('RandomForest', {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 2}),
    ('RandomForest', {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2}),
    ('RandomForest', {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 5}),  
    ('RandomForest', {'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 2}),
    ('RandomForest', {'n_estimators': 150, 'max_depth': 15, 'min_samples_split': 5}), 
    ('RandomForest', {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 2}),
    
    # XGBoost variants
    ('XGBoost', {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}),
    ('XGBoost', {'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.1}),
    ('XGBoost', {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05}),
    
    # Gradient Boosting variants  
    ('GradientBoosting', {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.1}),
    ('GradientBoosting', {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}),
    
    # Ridge (baseline)
    ('Ridge', {'alpha': 1.0}),
]

print(f"Test edilecek model sayısı: {len(model_configs)}")

Test edilecek model sayısı: 14


In [91]:
# Paralel model eğitimi
def parallel_model_training():
    futures = [
        train_and_evaluate.remote(
            model_name, params,
            X_train_ref, y_train_ref,
            X_test_ref, y_test_ref
        )
        for model_name, params in model_configs
    ]
    return ray.get(futures)

print("Paralel model eğitimi başlıyor...")
print(f"{len(model_configs)} model aynı anda eğitiliyor!\n")

model_results = benchmark(parallel_model_training, 'parallel_model_comparison')

Paralel model eğitimi başlıyor...
14 model aynı anda eğitiliyor!



(pid=gcs_server) [2025-12-10 08:31:18,753 E 18750 18750] (gcs_server) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-12-10 08:31:21,472 E 18865 18865] (raylet) main.cc:979: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(train_and_evaluate pid=18929) [2025-12-10 08:31:24,852 E 18929 19039] core_worker_process.cc:837: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14



✓ parallel_model_comparison
  Süre: 176.807 saniye
  Bellek: 0.84 MB


In [92]:
# Sonuçları göster
print("\nMODEL KARŞILAŞTIRMA SONUÇLARI")
print("="*90)
print(f"{'Model':<20} {'Params':<35} {'RMSE ($)':<12} {'MAE ($)':<12} {'R²':<10} {'Süre (s)':<10}")
print("-"*90)

sorted_results = sorted(model_results, key=lambda x: x['rmse'])
for r in sorted_results:
    params_str = str(r['params'])[:33]
    print(f"{r['model']:<20} {params_str:<35} {r['rmse']:<12.2f} {r['mae']:<12.2f} {r['r2']:<10.4f} {r['duration']:<10.1f}")

best_model = sorted_results[0]
print(f"\n{'='*90}")
print(f"EN İYİ MODEL: {best_model['model']}")
print(f"Parametreler: {best_model['params']}")
print(f"RMSE: ${best_model['rmse']:.2f}, R²: {best_model['r2']:.4f}")


MODEL KARŞILAŞTIRMA SONUÇLARI
Model                Params                              RMSE ($)     MAE ($)      R²         Süre (s)  
------------------------------------------------------------------------------------------
XGBoost              {'n_estimators': 150, 'max_depth'   3.57         1.66         0.9532     7.5       
XGBoost              {'n_estimators': 200, 'max_depth'   3.60         1.69         0.9525     10.5      
RandomForest         {'n_estimators': 150, 'max_depth'   3.62         1.71         0.9518     174.6     
RandomForest         {'n_estimators': 100, 'max_depth'   3.62         1.71         0.9518     122.2     
GradientBoosting     {'n_estimators': 100, 'max_depth'   3.63         1.74         0.9516     93.3      
XGBoost              {'n_estimators': 100, 'max_depth'   3.68         1.75         0.9503     4.0       
RandomForest         {'n_estimators': 200, 'max_depth'   3.69         1.80         0.9500     162.0     
RandomForest         {'n_estimators': 

## 6. Ray ile Hyperparameter Tuning (En İyi Model)

In [93]:
# En iyi model tipi için detaylı hyperparameter search
best_model_type = best_model['model']

if best_model_type == 'XGBoost':
    tuning_configs = [
        {'n_estimators': n, 'max_depth': d, 'learning_rate': lr, 'subsample': ss}
        for n in [100, 150, 200]
        for d in [5, 7, 9]
        for lr in [0.05, 0.1]
        for ss in [0.8, 1.0]
    ]
elif best_model_type == 'RandomForest':
    tuning_configs = [
        {'n_estimators': n, 'max_depth': d, 'min_samples_split': ms}
        for n in [100, 150, 200]
        for d in [10, 15, 20]
        for ms in [2, 5, 10]
    ]
else:
    tuning_configs = [
        {'n_estimators': n, 'max_depth': d, 'learning_rate': lr}
        for n in [50, 100, 150]
        for d in [3, 5, 7]
        for lr in [0.05, 0.1, 0.2]
    ]

print(f"\n{best_model_type} için {len(tuning_configs)} parametre kombinasyonu test edilecek")


XGBoost için 36 parametre kombinasyonu test edilecek


In [94]:
# Detaylı hyperparameter tuning
def detailed_hyperparameter_tuning():
    futures = [
        train_and_evaluate.remote(
            best_model_type, params,
            X_train_ref, y_train_ref,
            X_test_ref, y_test_ref
        )
        for params in tuning_configs
    ]
    return ray.get(futures)

print(f"Paralel hyperparameter tuning başlıyor...")
tuning_results = benchmark(detailed_hyperparameter_tuning, 'hyperparameter_tuning')

Paralel hyperparameter tuning başlıyor...

✓ hyperparameter_tuning
  Süre: 28.662 saniye
  Bellek: 0.57 MB


In [95]:
# En iyi sonuçları göster
sorted_tuning = sorted(tuning_results, key=lambda x: x['rmse'])[:10]

print(f"\n{best_model_type} - EN İYİ 10 PARAMETRE KOMBİNASYONU")
print("="*80)
for i, r in enumerate(sorted_tuning, 1):
    print(f"{i}. RMSE: ${r['rmse']:.3f}, R²: {r['r2']:.4f}")
    print(f"   Params: {r['params']}")

final_best = sorted_tuning[0]
print(f"\n{'='*80}")
print(f"FİNAL EN İYİ: RMSE=${final_best['rmse']:.3f}, R²={final_best['r2']:.4f}")


XGBoost - EN İYİ 10 PARAMETRE KOMBİNASYONU
1. RMSE: $3.566, R²: 0.9533
   Params: {'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.05, 'subsample': 1.0}
2. RMSE: $3.567, R²: 0.9532
   Params: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 1.0}
3. RMSE: $3.570, R²: 0.9532
   Params: {'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 1.0}
4. RMSE: $3.574, R²: 0.9531
   Params: {'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.1, 'subsample': 1.0}
5. RMSE: $3.578, R²: 0.9530
   Params: {'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.1, 'subsample': 1.0}
6. RMSE: $3.580, R²: 0.9529
   Params: {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1, 'subsample': 1.0}
7. RMSE: $3.581, R²: 0.9529
   Params: {'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.05, 'subsample': 1.0}
8. RMSE: $3.596, R²: 0.9525
   Params: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'subsample': 1.0}
9. RMSE: $3.596, 

## 7. Polars ile Hızlı Batch Prediction

In [96]:
# Final modeli eğit
if best_model_type == 'XGBoost':
    final_model = XGBRegressor(**final_best['params'], random_state=42, n_jobs=-1, verbosity=0)
elif best_model_type == 'RandomForest':
    final_model = RandomForestRegressor(**final_best['params'], random_state=42, n_jobs=-1)
else:
    final_model = GradientBoostingRegressor(**final_best['params'], random_state=42)

final_model.fit(X_train, y_train)
print(f"Final model eğitildi: {best_model_type}")

Final model eğitildi: XGBoost


In [97]:
# Polars ile yeni veri için batch prediction
def polars_batch_prediction():
    # Aynı ayın verisinden farklı sample al
    new_data = (
        pl.scan_parquet(taxi_files[0])  # Ocak 2023
        .select([
            'tpep_pickup_datetime',
            'trip_distance',
            'PULocationID',
            'DOLocationID',
            'passenger_count',
            'fare_amount'
        ])
        .filter(
            (pl.col('fare_amount') > 0) &
            (pl.col('trip_distance') > 0) &
            (pl.col('passenger_count') > 0) &
            (pl.col('passenger_count').is_not_null())
        )
        .with_columns([
            pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour'),
            pl.col('tpep_pickup_datetime').dt.weekday().alias('pickup_dayofweek'),
            (pl.col('tpep_pickup_datetime').dt.weekday() >= 5).cast(pl.Int8).alias('is_weekend'),
            (
                ((pl.col('tpep_pickup_datetime').dt.hour() >= 7) & 
                 (pl.col('tpep_pickup_datetime').dt.hour() <= 9)) |
                ((pl.col('tpep_pickup_datetime').dt.hour() >= 17) & 
                 (pl.col('tpep_pickup_datetime').dt.hour() <= 19))
            ).cast(pl.Int8).alias('is_rush_hour'),
        ])
        .head(100_000)  # İlk 100K
        .collect()
    )
    
    # Feature matrix
    X_new = new_data.select(feature_cols).to_numpy()
    
    # Tahmin
    predictions = final_model.predict(X_new)
    
    # Sonuçları Polars DataFrame'e ekle
    result = new_data.with_columns([
        pl.Series('predicted_fare', predictions)
    ]).with_columns([
        (pl.col('fare_amount') - pl.col('predicted_fare')).alias('prediction_error')
    ])
    
    return result

predictions_df = benchmark(polars_batch_prediction, 'batch_prediction')
print(f"\nTahmin yapılan satır: {len(predictions_df):,}")


✓ batch_prediction
  Süre: 0.274 saniye
  Bellek: 1.10 MB

Tahmin yapılan satır: 100,000


In [98]:
# Tahmin sonuçları
print("\nTAHMİN SONUÇLARI ÖRNEĞİ")
print("="*80)
predictions_df.select([
    'trip_distance', 'pickup_hour', 'fare_amount', 
    'predicted_fare', 'prediction_error'
]).head(10)


TAHMİN SONUÇLARI ÖRNEĞİ


trip_distance,pickup_hour,fare_amount,predicted_fare,prediction_error
f64,i8,f64,f32,f64
0.97,0,9.3,8.063667,1.236333
1.1,0,7.9,7.989832,-0.089832
2.51,0,14.9,14.37742,0.52258
1.43,0,11.4,10.483186,0.916814
1.84,0,12.8,12.477262,0.322738
1.66,0,12.1,10.862745,1.237255
11.7,0,45.7,46.646633,-0.946633
2.95,0,17.7,16.641829,1.058171
3.01,0,14.9,16.49785,-1.59785


In [74]:
# Prediction istatistikleri
pred_stats = predictions_df.select([
    pl.col('prediction_error').abs().mean().alias('MAE'),
    pl.col('prediction_error').abs().median().alias('Median_AE'),
    (pl.col('prediction_error')**2).mean().sqrt().alias('RMSE'),
    pl.col('prediction_error').mean().alias('Mean_Error'),
])

print("\nBATCH PREDICTION İSTATİSTİKLERİ")
print(pred_stats)


BATCH PREDICTION İSTATİSTİKLERİ
shape: (1, 4)
┌──────────┬───────────┬──────────┬────────────┐
│ MAE      ┆ Median_AE ┆ RMSE     ┆ Mean_Error │
│ ---      ┆ ---       ┆ ---      ┆ ---        │
│ f64      ┆ f64       ┆ f64      ┆ f64        │
╞══════════╪═══════════╪══════════╪════════════╡
│ 2.100425 ┆ 1.141557  ┆ 5.521032 ┆ 0.018744   │
└──────────┴───────────┴──────────┴────────────┘


## 8. Performans Özeti

In [75]:
# Sonuçları kaydet
results['total_memory_mb'] = round(get_memory_mb(), 2)
results['total_memory_gb'] = round(get_memory_mb() / 1024, 2)
results['row_count'] = int(total_rows)
results['best_model'] = best_model_type
results['best_params'] = final_best['params']
results['best_rmse'] = round(final_best['rmse'], 3)
results['best_r2'] = round(final_best['r2'], 4)
results['models_tested'] = len(model_configs) + len(tuning_configs)

os.makedirs('results', exist_ok=True)
with open('results/polars_ray_benchmark.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Sonuçlar kaydedildi: results/polars_ray_benchmark.json")

Sonuçlar kaydedildi: results/polars_ray_benchmark.json


In [76]:
print("\n" + "="*70)
print("POLARS + RAY HİBRİT PİPELİNE ÖZET")
print("="*70)

print("\n📊 VERİ")
print(f"   Toplam satır: {total_rows:,}")
print(f"   Dosya boyutu: {total_size / 1024**2:.1f} MB")

print("\n⚡ POLARS PERFORMANSI")
polars_ops = ['polars_lazy_scan', 'polars_aggregations', 'polars_feature_engineering']
for op in polars_ops:
    if op in results['operations']:
        print(f"   {op}: {results['operations'][op]['duration_sec']:.2f}s")

print("\n🚀 RAY PERFORMANSI")
ray_ops = ['parallel_model_comparison', 'hyperparameter_tuning']
for op in ray_ops:
    if op in results['operations']:
        print(f"   {op}: {results['operations'][op]['duration_sec']:.2f}s")

print("\n🏆 EN İYİ MODEL")
print(f"   Algoritma: {best_model_type}")
print(f"   RMSE: ${final_best['rmse']:.2f}")
print(f"   R²: {final_best['r2']:.4f}")
print(f"   Parametreler: {final_best['params']}")

print("\n📈 TOPLAM")
total_time = sum(op['duration_sec'] for op in results['operations'].values())
print(f"   Toplam süre: {total_time:.1f}s")
print(f"   Test edilen model: {results['models_tested']}")
print(f"   Bellek kullanımı: {results['total_memory_gb']:.2f} GB")


POLARS + RAY HİBRİT PİPELİNE ÖZET

📊 VERİ
   Toplam satır: 3,066,766
   Dosya boyutu: 606.3 MB

⚡ POLARS PERFORMANSI
   polars_lazy_scan: 0.00s
   polars_aggregations: 0.15s
   polars_feature_engineering: 0.15s

🚀 RAY PERFORMANSI
   parallel_model_comparison: 178.09s
   hyperparameter_tuning: 28.44s

🏆 EN İYİ MODEL
   Algoritma: XGBoost
   RMSE: $3.76
   R²: 0.9479
   Parametreler: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 1.0}

📈 TOPLAM
   Toplam süre: 207.2s
   Test edilen model: 50
   Bellek kullanımı: 6.15 GB


In [77]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    POLARS + RAY KOMBİNASYONU                         ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  POLARS (Veri İşleme)                                               ║
║  ├─ Lazy evaluation → Sadece gerekli veri okunur                   ║
║  ├─ Predicate pushdown → Filtreler dosya seviyesinde               ║
║  ├─ Column projection → Sadece gerekli sütunlar                    ║
║  └─ Paralel execution → Tüm CPU'lar kullanılır                     ║
║                                                                      ║
║  RAY (ML Pipeline)                                                  ║
║  ├─ @ray.remote → Fonksiyonları paralel çalıştır                   ║
║  ├─ Object store → Zero-copy veri paylaşımı                        ║
║  ├─ Task scheduling → Otomatik iş dağılımı                         ║
║  └─ Scalability → Cluster'a kolayca scale                          ║
║                                                                      ║
║  BİRLİKTE                                                           ║
║  ├─ Polars: Veri yükle, temizle, feature engineering               ║
║  ├─ NumPy: ML kütüphaneleri için dönüşüm                          ║
║  ├─ Ray: Model eğitimi ve hyperparameter tuning                    ║
║  └─ Polars: Batch prediction ve sonuç analizi                      ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")


╔══════════════════════════════════════════════════════════════════════╗
║                    POLARS + RAY KOMBİNASYONU                         ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  POLARS (Veri İşleme)                                               ║
║  ├─ Lazy evaluation → Sadece gerekli veri okunur                   ║
║  ├─ Predicate pushdown → Filtreler dosya seviyesinde               ║
║  ├─ Column projection → Sadece gerekli sütunlar                    ║
║  └─ Paralel execution → Tüm CPU'lar kullanılır                     ║
║                                                                      ║
║  RAY (ML Pipeline)                                                  ║
║  ├─ @ray.remote → Fonksiyonları paralel çalıştır                   ║
║  ├─ Object store → Zero-copy veri paylaşımı                        ║
║  ├─ Task scheduling → Otomatik iş dağılımı                    

In [78]:
# Ray'i kapat
ray.shutdown()
print("Ray kapatıldı.")

Ray kapatıldı.


---

## Sonuç

Bu notebook'ta **Polars + Ray** kombinasyonunun gücünü gördük:

| Bileşen | Görev | Avantaj |
|---------|-------|--------|
| **Polars** | Veri yükleme | Lazy evaluation, predicate pushdown |
| **Polars** | Feature engineering | Expression API, paralel execution |
| **Ray** | Model training | @ray.remote ile paralellestirme |
| **Ray** | Hyperparameter tuning | Tüm kombinasyonlar aynı anda |
| **Polars** | Batch prediction | Hızlı tahmin ve analiz |

**Gerçek dünya senaryolarında bu kombinasyon:**
- Büyük veri setlerinde bellek tasarrufu
- Model geliştirme süresini kısaltma
- Production-ready pipeline oluşturma

için idealdir!